In [ ]:
# For creating the dataset in the right format as required by the Yolo Model
!python create_csv.py
!python transfering_images.py
!python get_img_width_height.py
!python create_labels.py
!python create_bounding_box_csv.py
!python Create_yolov5_dataset.py
!python creatingyamlfile.py

In [ ]:
import os 
from os.path import join
from glob import glob
import yaml
import cv2
import matplotlib.pyplot as plt
import csv
import pandas as pd

classes = ['text']
cwd = './'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('../input/dataset-text-detect-gray/dataset_yolo_gray/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('../input/dataset-text-detect-gray/dataset_yolo_gray/images/valid/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 1,
    names = classes
    )

with open(join( cwd , 'text.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'text.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# Install W&B 
!pip install -q --upgrade wandb
# Login 
import wandb
wandb.login()

In [ ]:
! pip install -r yolov5/requirements.txt  # install

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!WANDB_MODE=online python ./yolov5/train.py --img 512 --batch 32 --epochs 30 --data ./text.yaml --weights '' --cfg ./yolov5/models/yolov5x.yaml --hyp ./yolov5/data/hyps/hyp.scratch.yaml --save_period 1 --project MENU_DIGITIZATION_Project --name extra-large-new-run-kaggle-grayscale

In [ ]:
!WANDB_MODE=online python ./yolov5/detect.py --weights ../input/dataset-yolo/Best_Model.pt --source ../input/dataset-yolo/dataset_yolo/dataset_yolo/test/ --img 1028 --conf 0.35 --save-txt

In [ ]:

image_paths = glob('./runs/detect/exp/labels/*')

In [ ]:

with open('./output.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Image_Name', 'Points', 'Labels'])
    for image_path in image_paths:
        with open(image_path) as f:
            Image_Name = image_path.split('/')[-1].split('.')[0]
            for bbox in f.read().split('\n'):
                points = bbox.split()[1:]
                writer.writerow([Image_Name, points, 0])

In [ ]:
output = pd.read_csv('./output.csv')
output.head()

In [ ]:
hashmapuh_h = {}
hashmapuh_w = {}
test_images = glob('../input/dataset-yolo/dataset_yolo/dataset_yolo/test/*')
for image_path in test_images:
    image_name = image_path.split('/')[-1].split('.')[0]
    image = cv2.imread(image_path)
    image_h, image_w, channel = image.shape
    hashmapuh_h[image_name] = image_h  
    hashmapuh_w[image_name] = image_w

In [ ]:
output['Image_Height'] = output['Image_Name'].map(hashmapuh_h)
output['Image_Width'] = output['Image_Name'].map(hashmapuh_w)

In [ ]:
import ast
output['Points'] = output['Points'].apply(lambda x: ast.literal_eval(x))

In [ ]:
import ast
def unnormalized(points, image_h, image_w):
    if points == []:
        return 'x'
    point_x = float(points[0])
    point_y = float(points[1])
    width = float(points[2])
    height = float(points[3])
    point_x *= image_w
    point_y *= image_h
    width *= image_w
    height *= image_h
    #return [point_x, point_y, width, height]
    return f'[{point_x}, {point_y}, {width}, {height}]'
    
output['point'] = 1
for i in range(len(output)):
    output['point'][i] = unnormalized(output['Points'][i], output['Image_Height'].iloc[i], output['Image_Width'].iloc[i] )

In [ ]:
def four_p_to_four_points(point):
    if point == 'x':
        return 'x'
    point = ast.literal_eval(point)
    x, y, w, h = point
    x = float(x)
    y = float(y)
    w = float(w)
    h = float(h)
    l = x - w / 2
    r = x + w / 2
    t = y - h / 2
    b = y + h / 2
    return [[l,t],[r,t],[r,b],[l,b]]

In [ ]:
output['point'] = output['point'].apply(four_p_to_four_points)

In [ ]:
output.to_csv('output_4_points.csv', index=False)